In [1]:
import tensorflow as tf 
import os
from glob import glob
pos_max=40
nb_channels=3
img_height,img_width=64,512
factor=32
BATCH_SIZE=32
def data_input_fn(recs,mode): 
    '''
      This Function generates data from gcs
      * The parser function should look similiar now because of datasetEDA
    '''
    def _parser(example):   
        feature ={  'image'  : tf.io.FixedLenFeature([],tf.string) ,
                    'mask'   : tf.io.FixedLenFeature([],tf.string) ,
                    'label'  : tf.io.FixedLenFeature([pos_max],tf.int64)
                            }    
        parsed_example=tf.io.parse_single_example(example,feature)
        # image
        image_raw=parsed_example['image']
        image=tf.image.decode_png(image_raw,channels=nb_channels)
        image=tf.cast(image,tf.float32)/255.0
        image=tf.reshape(image,(img_height,img_width,nb_channels))
        
        # mask
        mask_raw=parsed_example['mask']
        mask=tf.image.decode_png(mask_raw,channels=1)
        mask=tf.cast(mask,tf.float32)
        mask=tf.reshape(mask,(img_height,img_width,nb_channels))
        mask=tf.image.resize(mask,[img_height//factor,img_width//factor],"nearest")
        
        # label
        label=parsed_example['label']
            
        # position
        pos=tf.range(0,pos_max)
        pos=tf.cast(pos,tf.int32)
        
        return {"image":image,"label":tf.cast(label, tf.int32),"pos":pos,"mask":mask},tf.cast(label, tf.float32)
        # fixed code (for almost all tfrec training)
    dataset = tf.data.TFRecordDataset(recs)
    dataset = dataset.map(_parser)
    dataset = dataset.shuffle(128,reshuffle_each_iteration=True)
    dataset = dataset.repeat()
    dataset = dataset.batch(BATCH_SIZE,drop_remainder=True)
    dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
    dataset = dataset.apply(tf.data.experimental.ignore_errors())
    return dataset

rec_dir="/home/ansary/WORK/Work/APSIS/datasets/Recognition/datasets/bw/temp/tfrecord/"
recs=[rec_path for rec_path in glob(os.path.join(rec_dir,"*.tfrecord"))]
print(recs)
ds=data_input_fn(recs,"test")

['/home/ansary/WORK/Work/APSIS/datasets/Recognition/datasets/bw/temp/tfrecord/2.tfrecord', '/home/ansary/WORK/Work/APSIS/datasets/Recognition/datasets/bw/temp/tfrecord/0.tfrecord', '/home/ansary/WORK/Work/APSIS/datasets/Recognition/datasets/bw/temp/tfrecord/1.tfrecord']


In [2]:
import numpy as np
import matplotlib.pyplot as plt
print("---------------------------------------------------------------")
print("visualizing data")
print("---------------------------------------------------------------")
for x,y in ds.take(1):
    data=np.squeeze(x["image"][0])
    plt.imshow(data)
    plt.show()
    print("---------------------------------------------------------------")
    print("label:",x["label"][0])
    print("---------------------------------------------------------------")
    print("pos:",x["pos"][0])
    print("---------------------------------------------------------------")
    print("mask:",x["mask"][0][0])
    print("---------------------------------------------------------------")
    print('Image Batch Shape:',x["image"].shape)
    print('Label Batch Shape:',x["label"].shape)
    print('Position Batch Shape:',x["pos"].shape)
    print('Mask Batch Shape:',x["mask"].shape)
    print("---------------------------------------------------------------")
    print('Target Batch Shape:',y.shape)

---------------------------------------------------------------
visualizing data
---------------------------------------------------------------


In [ ]:
for x,y in ds.take(1):
    print(x.shape)